## OND'U管理


In [ ]:
import pandas as pd
import numpy as np
import datetime
# from dateutil.relativedelta import relativedelta

In [ ]:
useday = 0
# n日前のデータを使いたいなら、useday=nにする
# (例)昨日のデータを使いたいなら、useday=1にする

In [ ]:
def dt_change(dt):
    y = '{:0=4}'.format(dt.year)
    m = '{:0=2}'.format(dt.month)
    d = '{:0=2}'.format(dt.day)
    return f'{y}{m}{d}'

if useday == 0:
    dt_end = datetime.datetime.now()
else:
    dt_end = datetime.datetime.now() - datetime.timedelta(days=useday)
    
dt_start = dt_end - datetime.timedelta(days=30)
dt_start_ = dt_end - datetime.timedelta(days=13)   

# start = '20211011'
# end = '20211024'

In [ ]:
# 30日前
start = dt_change(dt_start)
#14日前
start_ = dt_change(dt_start_)
#開始日
end = dt_change(dt_end)

In [ ]:
remove_days = []

for i in range(30):
    now  = dt_start + datetime.timedelta(days=i)
    if now == dt_start_:
        break
    else:
        remove_days.append(now)   

In [ ]:
path1 = f'学生支部-{start}-{end}.csv'
path2 = f'社会人支部-{start}-{end}.csv'
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
dfs = [df1, df2]

In [ ]:
# 日付データの処理
def convert_date(df, col):
    df = df.copy()
    df[col] = pd.to_datetime(df[col])
    #df['日付'] = df[col].map(lambda x:str(x.year)+str(x.month).zfill(2)+str(x.day).zfill(2))
    df['日付'] = df[col].map(lambda x:str(x.month)+'/'+str(x.day))
    
    #df['月'] = df[col].map(lambda
    # df['日'] = df[col].map(lambda x:x.day)
    df = df.drop(col, axis=1)
    return df

In [ ]:
def data_shaping(df):
    df_ = df.copy()
    
    # 不要なカラムを削除
    drops = ["'ID'", "'グループ名'", "'発熱'", "'平熱'", "'最終出勤日'", 
         "'体温(問診票情報)'", "'症状(問診票情報)'", 
         "'通勤手段（選択）'", "'通勤手段（詳細）'",
        "'受診医療機器名'", "'PCR検査（有無）'", 
         "'PCR検査（詳細）'", "'感染者と接触有無'", 
         "'感染者接触（詳細）'", "'県外移動（有無）都道府県外移動、海外渡航 履歴'",
         "'県外移動（詳細：誰が）'", "'県外移動（詳細：場所）'", 
        "'県外移動（期間）'", "'人の過密場所への移動(有無)'", 
         "'人の過密場所への移動（詳細：誰が）'",
        "'人の過密場所への移動（詳細：場所）'", 
         "'人の過密場所への移動(期間表示）'"]
    
    df_ = df_.drop(drops, axis=1)
    
    # カラムの名前を変更
    renames = {"'氏名'":"氏名", 
           "'日付・時刻'":"日付・時刻",
           "'体温(体調情報)'":"体温",
           "'症状(体調情報)'":"症状",
           "'メッセージ'":"メッセージ"}
    df_ = df_.rename(columns=renames)
    
    #日付・時刻→日付
    df_ = convert_date(df_, '日付・時刻')
    
    # 引用符をなくす
    cols = ['氏名', '体温', '症状', 'メッセージ']
    for c in cols:
        df_[c] = df_[c].str[1:]
        df_[c] = df_[c].str[:-1]
        
    # 過去14日分以外のデータ(つまり、前から17日分のデータ)を削除
    
    for i in remove_days:
        now = f'{i.month}/{i.day}'
        df_ = df_[df_['日付'] != now]

    return df_

In [ ]:
df1 = data_shaping(df1)
df2 = data_shaping(df2)

#管理者
managers = ['管理　太郎', '管理　二郎', '管理　三郎']

for m in managers:
    for i in range(len(df2)):
        if df2.iloc[i]['氏名'] == m:
            df2 = df2[df2['氏名'] != m]
            break

df_ = pd.concat([df1, df2])

In [ ]:
# 省略しています
members = [
'苗字 一郎'
'苗字 二郎'
'苗字 三郎'
'苗字 四郎'
'苗字 五郎'
'苗字 六郎'
'苗字 七郎']


In [ ]:
for m in managers:
    for i in range(len(df2)):
        if df2.iloc[i]['氏名'] == m:
            df2 = df2[df2['氏名'] != m]
            break

In [ ]:
# 学年ごとの人数
#1,2,3,4年,院生,社会人
n1, n2, n3, n4, n5, n6 = 10,15,15,20,20,20

In [ ]:
#名前×日付データ
def name_day(df_, members=members):
    # 名簿
#     members = []
#     for m in df_['氏名'].unique():
#         members.append(m)

    # 日付一覧
    days = []
    for d in df_['日付'].unique():
        days.append(d)

    # http://hxn.blog.jp/archives/11591404.html
    A = pd.DataFrame(index=members, columns=days)

    # 37.5度以上があったら、体温表示、無ければ〇
    for m in members:
        personal_df = df_[df_.氏名==m]
        
        for i in range(len(personal_df)):
            # 高熱の場合は★をつける
            if float(personal_df.iloc[i]['体温']) > 37.4:
                personal_df.iloc[i]['体温'] = '★' + personal_df.iloc[i]['体温']

            
        for d in A.columns:
            day_data = personal_df[personal_df.日付==d]

            if day_data.empty is True:
                pass

            else:
                text = day_data.iloc[0]['体温']
                A.loc[m][d] = text   
            
    return A

In [ ]:
A = name_day(df_)

def highlight_negative(val):
    if val > 37.4:
        return 'color: {0}; font-weight: bold'.format('red')
    else:
        return 'color: {0}'.format('black')

A.style.applymap(highlight_negative)
A

In [ ]:
A.insert(0, '未記録数','0')
#A

In [ ]:
a = A.isnull().sum(axis=1)
for i in range(len(a)):
    b = a.iloc[i]
    if b == 0:
        A.iat[i,0] = '〇'
    else:
        A.iat[i,0] = b
#A

In [ ]:
# 学年ごとの人数
#1,2,3,4年,院生,社会人
n1, n2, n3, n4, n5, n6 = 10,15,15,20,20,20

In [ ]:
# insertは、列には使えるが行には使えないので、
# 転置→insert→転置する必要がある
n0_ = 0
n1_ = n1 + n0_
n2_ = n2 + n1_
n3_ = n3 + n2_
n4_ = n4 + n3_
n5_ = n5 + n4_

B = A.copy()
AT = A.T
AT.insert(n0_  , '大学1年', '----')
AT.insert(n1_+1, '大学2年', '----')
AT.insert(n2_+2, '大学3年', '----')
AT.insert(n3_+3, '大学4年', '----')
AT.insert(n4_+4, '大学院生', '----')
AT.insert(n5_+5, '社会人', '----')
A = AT.T


In [ ]:
n6_ = n6 + 1
A1 = A.iloc[:-n6_]
A2 = A.iloc[-n6_:]
# A1

In [ ]:
# pip install openpyxl
# pip install xlwt

In [ ]:
term = f'{start_}_{end}'
path = f'体温記録_{term}.xlsx'

In [ ]:
with pd.ExcelWriter(path) as writer:
    A1.to_excel(writer, sheet_name=f"学生支部")
    A2.to_excel(writer, sheet_name=f"社会人支部")

display(A)

In [ ]:
B1 = B.iloc[:n1_]
B2 = B.iloc[n1_:n2_]
B3 = B.iloc[n2_:n3_]
B4 = B.iloc[n3_:n4_]
B5 = B.iloc[n4_:n5_]
B6 = B.iloc[-n6:]
B45 = pd.concat([B4, B5])

In [ ]:
# B_list = [B1, B2, B3, B4, B5, B6]
B_list = [B1, B2, B3, B45, B6]


In [ ]:
def ok_missing(A, ALL_MISSING):  
    OK = []
    MISSING = []
    

    for i in A.index:
        n = A.loc[i].isna().sum()
        if n == 0:
            OK.append(i)
        elif n == 14:
            ALL_MISSING.append(i)
        else:
            text = f'{i}({n})'
            MISSING.append(text)
    
    return OK, MISSING

In [ ]:
def all_ok_missing(B_list):
    Os = []
    Ms = []
    ALL_MISSING = []
    for i in B_list:
        o, m = ok_missing(i, ALL_MISSING)
        Os.append(o)
        Ms.append(m)
    return Os, Ms, ALL_MISSING

In [ ]:
Os, Ms, ALL_MISSING = all_ok_missing(B_list)

In [ ]:
def ok_display(O):
    j = 0
    k = []
    for i in O:
        if j == 9:
            print(k)
            j = 0
            k = []

        k.append(i)
        j += 1
    print(k)

In [ ]:
texts = ['大学1年', '大学2年', '大学3年', '大学4年以上', '社会人']

In [1]:
print(f'本日の名簿({start_}-{end})')
print()
for i in range(len(texts)):
    print(texts[i], ':',len(Os[i]))
    ok_display(Os[i])
    print()

In [2]:
# 出していない人リストを表示するように変更
print(f'出してない人リスト({start_}-{end})')
print()
for i in range(len(texts)):
    print(texts[i], ':',len(Ms[i]))
    ok_display(Ms[i])
    print()
print()
print('2週間1回も出してない人')
ok_display(ALL_MISSING)